<a href="https://colab.research.google.com/github/EwFox76/IMERS-O-ALURA-IA/blob/main/Vers%C3%A3o_estavel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
# Instalar bibliotecas necessárias.
!pip install -q google-genai
!pip install -q google-genai ipywidgets
!pip install -q google-adk
print("Bibliotecas instaladas.")

Bibliotecas instaladas.


In [ ]:
# Importações Nescessárias.
import os
from google.adk.agents import Agent
print("Agent: OK")
from google.adk.runners import Runner
print("Runner: OK")
from google.adk.sessions import InMemorySessionService
print("In Memory: Ok")
from google.adk.tools import google_search
print("Google Search: OK")
from datetime import date
print("Date: OK")
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
print("API_KEY: OK")

import ipywidgets as widgets
from ipywidgets import HTML, Output, Label, Textarea, Button, VBox, HBox, Layout
from IPython.display import display
from datetime import datetime
print("Widgets: OK")

import google.adk as adk
print("Adk: OK")

from google import genai
client = genai.Client()
modelo = "gemini-2.5-flash-preview-04-17"
print("Modelo Gemini: OK")

from google.genai import types
print("Tipos para configurar os modelos: OK")



In [64]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [66]:
# agente de busca#
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar frase motivadoras, poemas reconfortantes, videos ou musicas com historias inspiradoras.
        Esses lançamentos relevantes podem ou não ser atuais, de no máximo dez anos antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos


In [82]:
# --- Agente motivador --- #
################################################
def agente_motivador(topico, lancamentos_buscados):
    motivador = Agent(
        name="agente_motivador",
        model="gemini-2.5-flash-preview-04-17",
        # Inserir as instruções do Agente motivador #################################################
        instruction="""
        Você é motivaciocal, especialista em inspirar as pessoas. Com base na lista de
        frase motivadoras, poemas reconfortantes e historias inspiradoras e relevantes buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para selecionar entre eles
        quais são os pontos mais relevantes que poderíamos usar para inspiras. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e filtrar os que são mais alegres.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos
        a serem apresentados posteriormente.
        """,
        description="Agente motivacional",
        tools=[google_search]
    )

    entrada_do_agente_motivador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_motiva = call_agent(motivador, entrada_do_agente_motivador)
    return plano_motiva

In [83]:
# --- Agente de correção --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
            Você é um Redator Criativo especializado em criar frases motivacionais, alegres e inspiradoras.
            Você escreve posts para as pessoas que querem uma palavra de conforto e motivação.
            Utilize o tema fornecido no plano motiva e os pontos mais relevantes fornecidos, e com base nisso,
            escreva um rascunho novo ou original para exibir um diferente sempre que a pessoa falar de um de seus sentimentos.
            O post deve ser inspirador, informativo, com linguagem simples.
            """,
        description="Agente redator de posts motivadores"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_motiva}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [84]:
# --- Agente de correção --- #
##########################################

def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts, frases, poesia, com foco inspirar.
            Por ter um público de varias idades, entre 12 e 60 anos, use um tom de escrita adequado.
            Revise o rascunho de post gerado abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom leve e alegre.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [80]:
# Instruções de sistema

chat_config = types.GenerateContentConfig (system_instruction = """
Você é um agente de suporte positivo e encorajador, projetado para oferecer palavras de conforto, dicas gerais de bem-estar e positividade para pessoas que podem estar passando por momentos difíceis.
É FUNDAMENTAL que você NUNCA aja como um profissional de saúde mental.
Suas respostas devem ser:
- Empáticas e acolhedoras.
- Focadas em positividade e encorajamento.
- Oferecer dicas GERAIS de bem-estar (ex: sobre sono, alimentação, exercícios leves, hobbies, relaxamento, familiar, lazer, amizade, hidratação, tempo ao ar livre).
- NUNCA fazer diagnósticos ou sugerir tratamentos ou medicamentos.
- SEMPRE incluir um DISCLAIRER CLARO no início ou final de CADA interação de que você não é um profissional e RECOMENDAR FORTEMENTE a busca por AJUDA PROFISSIONAL (psicólogo, psiquiatra).
- NUNCA responder a pedidos de ajuda imediata em crises ou pensamentos suicidas. Nesses casos, instrua a pessoa a procurar ajuda profissional imediatamente, ligar para serviços de emergência ou para um centro de valorização da vida (como o CVV no Brasil, ligando para 188).

Exemplos de coisas para dizer:
- "É corajoso compartilhar como você se sente."
- "Lembre-se de ser gentil consigo mesmo."
- "Pequenos passos fazem diferença."
- "Que tal tentar uma caminhada curta hoje, se possível?"
- "Respirar fundo por alguns minutos pode ajudar a relaxar."
- "Lembre-se que você não está sozinho(a)."
- "Tem pessoas que se importam com você"
- "A vida é um presente único"
- "Beber água e se alimentar bem ajuda no bem-estar geral."

Exemplos de coisas para NÃO dizer:
- "Você tem depressão." (Diagnóstico)
- "Tome este medicamento." (Tratamento)
- "Eu posso curar sua depressão." (Afirmação falsa)
- "Não se sinta assim." (Invalidando sentimentos)
- "É só uma fase, vai passar." (Minimizando)

O DISCLAIMER OBRIGATÓRIO ao final e no início da interação deve ser algo como:
"Olá! Sou um agente de IA de apoio positivo e não substituo a ajuda profissional.
Se você está enfrentando dificuldades significativas, por favor, procure a ajuda de um"
OU
"Lembre-se, sou uma IA e não posso oferecer aconselhamento médico ou psicológico.
Se você precisa de ajuda profissional, por favor, procure um especialista."

OU ALGO SEMELHANTE, MAS CLARO E RECOMENDANDO AJUDA PROFISSIONAL.

Vamos começar. Apresente-se brevemente com o disclaimer inicial.
"""
)
chat = client.chats.create(model=model, config=chat_config)

print("Instruções de sistema: OK")


Instruções de sistema: OK


In [85]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)

    plano_motiva = agente_motivador(topico, lancamentos_buscados)

    rascunho_gerado = agente_redator(topico, plano_motiva)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_gerado))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_gerado)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: triste
Maravilha! Vamos então criar o post sobre novidades em triste

--- 📝 Resultado do Agente 3 (Redator) ---



NameError: name 'to_markdown' is not defined

In [86]:
## Interface Interativa com Widgets

# Variável para controlar se a conversa está ativa
chat_active = True

# Widget de saída para exibir a conversa - Customized Layout and Style
output_area = Output(
    layout={
        'border': '4px solid black',
        'width': 'auto',  # Allow width to adjust
        'height': '300px', # Set a fixed height
        'overflow_y': 'auto' # Add scrollbar if content overflows
    },
    # Using style for background color
    style={
        'background_color': '#f0f0f0' # Light gray background
    }
)

# Widget de entrada para o usuário digitar - Customized Layout
input_area = Textarea(
    placeholder='Digite seus sentimentos aqui...',
    layout=Layout(width='auto', height='100px') # Changed width to auto
)

# Botão para enviar a mensagem - Customized Style
send_button = Button(
    description='Enviar',
    style={
        'button_color': 'darkgreen', # Change button color to light blue
        'text_color': 'black',      # Set text color to black
        'font_weight': 'bold'       # Set font weight to bold
    }
)

# Botão para sair da conversa - Customized Style
exit_button = Button(
    description='Sair da Conversa',
    style={
        'button_color': 'salmon', # Change button color to salmon
        'text_color': 'black',   # Set text color to black
        'font_weight': 'bold'    # Set font weight to bold
    }
)


# Caixa para organizar a entrada e o botão de envio horizontalmente
input_and_send_box = HBox([input_area, send_button]) # Renamed for clarity

# Caixa para organizar os botões de envio e sair na parte inferior
button_box = HBox([send_button, exit_button], layout=Layout(justify_content='space-between')) # New HBox for buttons

# Função para lidar com o clique no botão Enviar
def on_send_button_clicked(b):
    global chat_active
    if chat_active:
        user_input = input_area.value
        input_area.value = '' # Limpa a área de entrada após enviar

        if user_input.lower() == 'sair':
            on_exit_button_clicked(None) # Chama a função de sair se o usuário digitar "sair"
            return

        with output_area:
            print(f"Você: {user_input}\n")
            response = chat.send_message(user_input)
            print(f"Resposta: {response.text}\n")

# Função para lidar com o clique no botão Sair
def on_exit_button_clicked(b):
    global chat_active
    chat_active = False
    input_area.disabled = True
    send_button.disabled = True
    exit_button.disabled = True
    exit_button.description = "Saída Solicitada"
    with output_area:
        print("Conversa encerrada. Obrigado por compartilhar!")

# Conectar as funções aos botões
send_button.on_click(on_send_button_clicked)
exit_button.on_click(on_exit_button_clicked)

# Exibir a interface
data_de_hoje = date.today().strftime("%d/%m/%Y")
print("Bem-vindo! Clique no botão 'Sair da Conversa' a qualquer momento para encerrar.")
# Display the widgets in a VBox to stack them vertically
display(output_area, input_area, button_box) # Changed display order and components

# Opcional: Enviar uma mensagem inicial para o chatbot obter a apresentação com disclaimer
# (Isto pode ser adaptado dependendo de como você quer que a conversa inicie)
with output_area:
    # Você pode enviar um prompt inicial "Olá" ou algo similar aqui
    # para fazer o chatbot iniciar a conversa com o disclaimer.
    # Por exemplo:
    response = chat.send_message("Inicie a conversa")
    print(f"Resposta: {response.text}\n")

Bem-vindo! Clique no botão 'Sair da Conversa' a qualquer momento para encerrar.


Output(layout=Layout(border='4px solid black', height='300px', overflow_y='auto', width='auto'))

Textarea(value='', layout=Layout(height='100px', width='auto'), placeholder='Digite seus sentimentos aqui...')